In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime

import numpy as np
from operator import itemgetter

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM
from libs.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [6]:
from libs.Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 179 ( 0.5%) Users that have less than 1 test interactions


In [7]:
from libs.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

In [8]:
results = pd.DataFrame()

In [9]:
save_path = "result_experiments/SLIM/"

In [11]:
if os.path.exists(save_path):
    if os.path.exists(save_path+"results.csv"):
        results = pd.read_csv(save_path+"results.csv")
else:
    os.makedirs(save_path)

In [12]:
max_MAP = -1 

In [ ]:
best_param1 = {"topK": 300,
                "l1_ratio": 0.11777777777777779,
                "alpha": 0.00023333333333333333}

best_param2 = {"topK": 300,
                "l1_ratio": 0.112,
                "alpha": 0.000255}

In [21]:
np.linspace(0.0002, 0.00026, 4)

array([0.0002 , 0.00022, 0.00024, 0.00026])

In [ ]:
URM_train_coo = URM_train.tocoo()
slim_model = SLIMElasticNetRecommender(URM_train_coo, verbose=True)

for topK in [300, 310, 320, 330, 340, 350, 360, 370, 380]:
    for l1_ratio in np.linspace(0.111, 0.118, 5):
        for alpha in np.linspace(0.0002, 0.00026, 4):
            current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            params = {
                "topK": topK,
                "l1_ratio": l1_ratio,
                "alpha": alpha, 
                "time": current_timestamp
            }

            slim_model.fit(**{key: params[key] for key in ["l1_ratio", "alpha", "topK"]})

            metrics_df, _ = evaluator_validation.evaluateRecommender(slim_model)
            if max_MAP < metrics_df["MAP"][10]:
                max_MAP = metrics_df["MAP"][10]
            print(', '.join(f"{key}: {value}" for key, value in params.items()), "MAP: ", metrics_df["MAP"][10], "max MAP: ", max_MAP)
                  
            params_df = pd.DataFrame([params])
            iteration_results = pd.concat([params_df, metrics_df.reset_index()], axis=1)
            results = pd.concat([results, iteration_results], ignore_index=True)
            results.to_csv(save_path+"results.csv", index=False)

SLIMElasticNetRecommender: Processed 8610 (22.6%) in 5.00 min. Items per second: 28.69
SLIMElasticNetRecommender: Processed 17498 (45.9%) in 10.00 min. Items per second: 29.16
SLIMElasticNetRecommender: Processed 26265 (68.9%) in 15.00 min. Items per second: 29.18
SLIMElasticNetRecommender: Processed 35218 (92.4%) in 20.00 min. Items per second: 29.35
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 21.70 min. Items per second: 29.28
EvaluatorHoldout: Processed 35557 (100.0%) in 15.87 sec. Users per second: 2241
topK: 300, l1_ratio: 0.111, alpha: 0.0002, time: 2024-11-30 01:14:18 MAP:  0.06027638162913745 max MAP:  0.06027638162913745
SLIMElasticNetRecommender: Processed 8643 (22.7%) in 5.00 min. Items per second: 28.80
SLIMElasticNetRecommender: Processed 17380 (45.6%) in 10.00 min. Items per second: 28.96
SLIMElasticNetRecommender: Processed 26305 (69.0%) in 15.00 min. Items per second: 29.23
SLIMElasticNetRecommender: Processed 35332 (92.7%) in 20.00 min. Items per second: 29.

In [ ]:
But if I train model on all data, and then use it to evaluate on val dataset, I recieve aroun 0.13